# Innlevering 1 - DAT801

1. Her importeres nødvendige biblioteker.

In [ ]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
import xgboost as xgb

#from sklearn.pipeline import Pipeline
#from sklearn.model_selection import cross_val_predict, cross_val_score

2. Her settes lokal path.

In [ ]:
NB_DIR = Path.cwd()
DATA = NB_DIR/'data'
DATA.mkdir(exist_ok=True)

3. Her gjøres train- og test.csv om til pandas dataframes. Jeg velger med denne oppgaven å ikke kjøre en train_test_split(som jeg også da ville kjørt som stratified for å få en representativ fordeling mellom splittene), da vi har filene i 2 bestanddeler, selv om jeg da må score mot Kaggle underveis. Jeg synes bare det var enklere å forholde seg til.

In [ ]:
train = pd.read_csv(DATA/'train.csv')
test = pd.read_csv(DATA/'test.csv')

4. Inspiserer så datasettene. Ser manglende verdier i train, men ikke i test.

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.shape

In [ ]:
test.info()

5. Kikker på oversikten over høyeste representasjon av labels, samt ser på korrelasjoner mellom features, og mellom features og target.

In [ ]:
train["target"].hist() 
plt.show()

In [ ]:
corr = train.corr() #Sjekker korrelasjon mellom features for å eventuelt selektere bort features.
corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
corr_matrix = train.corr() #sjekker korrelasjon mellom features og target.
print(corr_matrix["target"].sort_values(ascending=False))

6. Om jeg fant høyt korrelerte features ville jeg vurdert å fjerne dem med funksjonen under. I dette tilfellet er korrelasjonene så lave, at jeg fjerner ikke noen, da jeg fikk dårligere score ved å gjøre det.

#Denne funksjonen fjerner features med korrelasjon over angitt tersel.

def remove_collinear_features(x, threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model 
        to generalize and improves the interpretability of the model.

    Inputs: 
        x: features dataframe
        threshold: features with correlations greater than this value are removed

    Output: 
        dataframe that contains only the non-highly-collinear features
    '''

    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i+1):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)

            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    print(drops)
    x = x.drop(columns=drops, inplace=True)

    return x

6. Her tilegnes features i X_train og id kolonnen blir droppet. Labels/targets lagres i y_train.

In [ ]:
X_train = train.drop(["id", "target"], axis = 1)

In [ ]:
y_train = train["target"].values

In [ ]:
X_test = test.drop(["id"], axis = 1)

7. Her imputeres manglende verdier i datasettet med strategien median verdi.

In [ ]:
imp = SimpleImputer(strategy = "median")

In [ ]:
X_train_imputed = imp.fit_transform(X_train)

In [ ]:
X_test_imputed = imp.transform(X_test)

8. Her skaleres det ferdig imputerte datasettet ved å bruke StandardScaler.

In [ ]:
std = StandardScaler()

In [ ]:
X_train_std = std.fit_transform(X_train_imputed)

In [ ]:
X_train_std = pd.DataFrame(data=X_train_std,columns=X_train.columns)

In [ ]:
X_test_std = std.transform(X_test_imputed)

9. Modellvalg - Jeg har valgt å bruke et voting ensemble for å forbedre treffsikkerheten på predikasjoner. Jeg har i andre notebooks testet med å bruke GridSearchCV sammen med RandomForestClassifier, og bruker derfor optimaliserte parametre på denne. Siden dette er ganske tidkrevende, velger jeg å bruke defaultverdier på de andre modellene, med unntak av random_state.

In [ ]:
rf = RandomForestClassifier(max_depth = 18, max_features= 5, min_samples_leaf = 3, min_samples_split = 11, n_estimators = 850,
 n_jobs = -1, random_state = 42)
gb = GradientBoostingClassifier(random_state = 42)
gbm = xgb.XGBClassifier(n_estimators= 2000, max_depth= 4, min_child_weight= 2, gamma=0.9,
subsample=0.8, colsample_bytree=0.8, objective= 'multi:softmax', scale_pos_weight=1, booster = "dart", n_jobs = -1)
#svc = SVC(random_state = 42, probability=True)
gnb = GaussianNB()

In [ ]:
models = [("rf", rf),
         ("gb", gb),
         ("gbm", gbm),
         ("gnb", gnb)]

10. Knytter valgte modeller til VotingClassifier og trener modellen.

In [ ]:
ensemble = VotingClassifier(models, voting="soft")

In [ ]:
ensemble.fit(X_train_std, y_train)

In [ ]:
#train_acc = accuracy_score(y_train, ensemble.predict(X_train_std))

In [ ]:
#print('Train Accuracy: {}'.format(train_acc))

11. Predikerer på testsettet basert på den trente modellen.

In [ ]:
y_pred = ensemble.predict(X_test_std)

In [ ]:
y_pred

12. Lager innleveringsfil ved å lage en pandas dataframe med id fra testfilen og predikasjoner fra esemblet, konvertert til en excelfil.

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'target': y_pred})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission_lkk_7.csv', index=False)

13. Final notes. Dersom dette hadde blitt levert som 1 datasett(1 fil, og ikke noe Kaggle konkurranse), ville jeg brukt train,test,split på datasettet. Jeg ville da ha rensket i nullverdier/korrelerte features og lignende før jeg hadde splittet opp datasettet. Hadde jeg ikke brukt voting ensemble(dette tok fryktelig lang tid å trene på dette datasettet. Etter den hadde stått i 2 dager avbrøt jeg den. Jeg har tydeligvis vært for detaljert i mine innstillinger på XGboost) ville jeg benyttet RandomSearch + GridSearchCV for å ivareta kryssvalidering og hyperparametertuning. Jeg ville da også hatt muligheten for å score underveis, samt laget en confusion matrix for å se på treffsikkerheten til modellen min med TP, FP, TN, FN. Med tanke på at jeg nå ikke har en y_test tilgjengelig å sammenligne med på måten jeg har løst oppgaven min, så får jeg ikke gjort dette. Hadde jeg hatt bedre tid, ville jeg også ha testet stacking av modeller sammen med en booster og trent modellen på nytt.